In [1]:
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,237 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,763 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Pac

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

def setup_driver():
    """Настройка headless Chrome (без интерфейса)"""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=chrome_options)
    return driver

def parse_table(driver):
    """
    Извлекает данные со страницы и возвращает список записей:
      [ [№, БИН/ИИН, Наименование субъекта здравоохранения, На заключение долгосрочного договора, Юридический адрес, ...], ... ]
    """
    data = []
    rows = driver.find_elements(By.CSS_SELECTOR, "table.table-bordered tbody tr")

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")

        # Проверяем, что строка содержит достаточное количество столбцов
        if len(cols) >= 15:
            reg_number = cols[0].text.strip()
            bin_iin = cols[1].text.strip()
            company_name = cols[2].text.strip()
            long_term_contract = cols[3].text.strip()
            legal_address = extract_multiline_data(cols[4].get_attribute('innerHTML'))
            ownership_form = cols[5].text.strip()
            region = cols[6].text.strip()
            date_inclusion = cols[7].text.strip()
            date_exclusion = cols[8].text.strip()
            last_change = cols[9].text.strip()
            medical_help_type = extract_multiline_data(cols[10].get_attribute('innerHTML'))
            sub_medical_help_type = extract_multiline_data(cols[11].get_attribute('innerHTML'))
            co_medical_help_type = cols[12].text.strip()
            co_sub_medical_help_type = cols[13].text.strip()
            status_healthcare_subject = cols[14].text.strip()
            status = cols[15].text.strip()

            data.append([
                reg_number,
                bin_iin,
                company_name,
                long_term_contract,
                legal_address,
                ownership_form,
                region,
                date_inclusion,
                date_exclusion,
                last_change,
                medical_help_type,
                sub_medical_help_type,
                co_medical_help_type,
                co_sub_medical_help_type,
                status_healthcare_subject,
                status
            ])

    return data

def extract_multiline_data(cell):
    """Заменяет тег <hr> на запятую и объединяет данные в одну строку"""
    # Заменяем все вхождения <hr style="margin: 7px"> на запятую
    return ', '.join([item.strip() for item in cell.replace('<hr style="margin: 7px">', ',').split(',') if item.strip()])

def main():
    # Стартовая страница
    start_url = "https://fms.ecc.kz/ru/fsms/healthcare_subjects"

    driver = setup_driver()
    driver.get(start_url)
    time.sleep(2)  # Пауза для загрузки страницы

    all_data = []
    visited_urls = set()

    while True:
        current_url = driver.current_url
        if current_url in visited_urls:
            # Цикл завершен, так как новых страниц нет
            break
        visited_urls.add(current_url)

        # Парсим таблицу
        page_data = parse_table(driver)
        all_data.extend(page_data)

        # Ищем кнопку ">"
        try:
            pagination = driver.find_element(By.CSS_SELECTOR, "ul.pagination")
            arrow = pagination.find_element(By.XPATH, ".//a[text()='>']")
            # Пробуем кликнуть
            arrow.click()
            time.sleep(2)  # Пауза между страницами
        except NoSuchElementException:
            print("Кнопка '>' не найдена, заканчиваем.")
            break
        except ElementNotInteractableException:
            print("Кнопка '>' не активна, заканчиваем.")
            break

    # Закрываем браузер
    driver.quit()

    # Формируем DataFrame с правильными названиями столбцов
    df = pd.DataFrame(all_data, columns=[
        "№",
        "БИН/ИИН",
        "Наименование субъекта здравоохранения",
        "На заключение долгосрочного договора",
        "Юридический адрес",
        "Форма собственности субъекта здравоохранения",
        "Регион, населению которого будут оказываться услуги",
        "Дата включения в базу данных",
        "Дата исключения из базы данных",
        "Дата последнего изменения",
        "Вид/Форма медицинской помощи (Поставщик)",
        "Подвида вида/формы медицинской помощи (Поставщик)",
        "Вид/Форма медицинской помощи (Соисполнитель)",
        "Подвида вида/формы медицинской помощи (Соисполнитель)",
        "Статус субъекта здравоохранения",
        "Статус"
    ])

    # Сохраняем в CSV и Excel
    df.to_csv("healthcare_subjects.csv", index=False, encoding="utf-8-sig")
    df.to_excel("healthcare_subjects.xlsx", index=False)
    print(f"Всего спарсили записей: {len(df)}")
    print("✅ Данные сохранены в healthcare_subjects.csv и healthcare_subjects.xlsx")

    return df

if __name__ == "__main__":
    df = main()
    # Если хотите посмотреть в Colab, раскомментируйте:
from IPython.display import display
display(df)


Кнопка '>' не найдена, заканчиваем.
Всего спарсили записей: 4513
✅ Данные сохранены в healthcare_subjects.csv и healthcare_subjects.xlsx


,№,БИН/ИИН,Наименование субъекта здравоохранения,На заключение долгосрочного договора,Юридический адрес,Форма собственности субъекта здравоохранения,"Регион, населению которого будут оказываться услуги",Дата включения в базу данных,Дата исключения из базы данных,Дата последнего изменения,Вид/Форма медицинской помощи (Поставщик),Подвида вида/формы медицинской помощи (Поставщик),Вид/Форма медицинской помощи (Соисполнитель),Подвида вида/формы медицинской помощи (Соисполнитель),Статус субъекта здравоохранения,Статус
0,66177,240740029785,"Товарищество с ограниченной ответственностью ""...",Нет,"Атырауская область, г.Атырау, Атырауская облас...",Частные организации,Атырауская область,2025-03-20 18:00:02,,2025-03-20 18:00:02,Специализированная медицинская помощь в стацио...,Стационарозамещающая хирургическая медицинская...,Консультативно-диагностическая помощь (специал...,,Поставщик / Соисполнитель,Включен в БД
1,66174,100140017375,"ТОO ""Centre Nova Diagnostic""",Нет,"г.Алматы, Медеуский район, г.Алматы, Медеуский...",Частные организации,Кызылординская область,2018-05-21 00:00:00,,2025-03-20 17:08:22,,,Консультативно-диагностическая помощь (специал...,,Соисполнитель,Включен в БД
2,66175,221140038117,"ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",Нет,"Кызылординская область, г.Кызылорда",Частные организации,Кызылординская область,2023-04-07 16:22:03,,2025-03-20 17:07:33,,,Консультативно-диагностическая помощь (специал...,,Соисполнитель,Включен в БД
3,66054,971040000944,"ТОО ""ФИРМА ЧА-КУР""",Нет,"Мангистауская область, г.Актау, Мангистауская ...",Частные организации,Мангистауская область,2017-09-12 00:00:00,,2025-03-20 15:24:31,Оказание медицинской помощи на уровне первично...,"Стимулирующий компонент подушевого норматива, ...",Консультативно-диагностическая помощь (специал...,,Поставщик / Соисполнитель,Включен в БД
4,66127,240740028291,"Товарищество с ограниченной ответственностью ""...",Нет,"Актюбинская область, г.Актобе, район Астана, А...",Частные организации,Актюбинская область,2025-03-20 14:13:07,,2025-03-20 14:13:07,,,Консультативно-диагностическая помощь (специал...,,Соисполнитель,Включен в БД
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4508,51,000841000509,"ТОО ""INVIVO"" (ИНВИВО)",Нет,г.Астана,Частные организации,г.Астана,2017-09-09 00:00:00,2020-11-10 10:33:18,2020-11-10 10:33:18,Консультативно-диагностические услуги,,Консультативно-диагностические услуги,,Поставщик / Соисполнитель,Исключен из БД
4509,81,010140002734,"ГКП на ПХВ ""Областной наркологический диспансер""",Нет,"Алматинская область, г.Қонаев",Государственные организации,Алматинская область,2018-04-28 00:00:00,2022-11-14 14:36:56,2022-11-14 14:36:56,"Медико-социальная помощь больным алкоголизмом,...",,"Медико-социальная помощь больным алкоголизмом,...",,Поставщик / Соисполнитель,Исключен из БД
4510,8,000140004769,КГП на ПХВ «Областной центр скорой медицинской...,Нет,"Северо-Казахстанская область, г.Петропавловск",Государственные организации,Северо-Казахстанская область,2019-06-11 00:00:00,2019-11-13 14:43:53,2019-11-13 14:43:53,"Скорая медицинская помощь, Медицинская помощь ...",,Скорая медицинская помощь\nМедицинская помощь ...,,Поставщик / Соисполнитель,Исключен из БД
4511,10,000140005221,Медицинское учреждение «Асар»,Нет,"Актюбинская область, г.Актобе",Частные организации,Актюбинская область,2017-09-13 00:00:00,2025-03-05 12:20:48,2025-03-05 12:20:48,"Стационарозамещающая медицинская помощь, Стаци...",,Стационарозамещающая медицинская помощь\nСтаци...,,Поставщик / Соисполнитель,Исключен из БД
